In [ ]:
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd

In [ ]:
# Get data and set working directory

arealdekke = gpd.GeoDataFrame()
bygninger = gpd.GeoDataFrame()
samferdsel = gpd.GeoDataFrame()

fylker_n50 = ['oslo', 'viken', 'innlandet', 'vestfold']       # n50 2020

year = 2020

for f in fylker_n50:

    data = f'./{f}'
    
    arealdekke_path = os.path.join(data, f"arealdekke_a_{year}.shp")
    bygninger_path = os.path.join(data, f"bygninger_a_{year}.shp")
    samferdsel_path = os.path.join(data, f"samferdsel_l_{year}.shp")

    arealdekke = pd.concat([arealdekke, gpd.read_file(arealdekke_path)])
    bygninger = pd.concat([bygninger, gpd.read_file(bygninger_path)])
    samferdsel = pd.concat([samferdsel, gpd.read_file(samferdsel_path)])

print(arealdekke.shape)
print(bygninger.shape)
print(samferdsel.shape)

In [ ]:

aoi = gpd.read_file("./32vnm.geojson")

from operator import itemgetter
bounds = aoi.geometry.apply(lambda x: x.bounds).tolist()

minx, miny, maxx, maxy = min(bounds, key=itemgetter(0))[0], min(bounds, key=itemgetter(1))[1], max(bounds, key=itemgetter(2))[2], max(bounds, key=itemgetter(3))[3]
bbox = [minx, miny, maxx, maxy]

print(bbox)

In [ ]:
arealdekke = gpd.clip(arealdekke, bbox)
bygninger = gpd.clip(bygninger, bbox)
samferdsel = gpd.clip(samferdsel, bbox)

In [ ]:
area_objtypes = list(arealdekke.objtype.unique())
print(area_objtypes)

bygg_objtypes = list(bygninger.objtype.unique())
print(bygg_objtypes)

samf_objtype = list(samferdsel.objtype.unique())
print(samf_objtype)

samf_objtype = list(samferdsel.vegkategor.unique())
print(samf_objtype)

samf_medium = list(samferdsel.medium.unique())
print(samf_medium)

In [ ]:
area_types = ['Tettbebyggelse', 'Industriområde', 'Steinbrudd', 'BymessigBebyggelse', 'Lufthavn', 'Steintipp']

area_vegetasjon_types = ['DyrketMark', 'Skog', 'Myr']
area_vann_types =['Innsjø','Havflate', 'Elv', 'InnsjøRegulert']
area_open_types = ['ÅpentOmråde']

area_objtypes.sort()
test_list = area_types+area_vegetasjon_types+area_vann_types+area_open_types
test_list.sort()
print(area_objtypes == test_list)

samf_veg = ['VegSenterlinje']
samf_veg_kategori = ['E', 'R', 'F', 'K']
samf_bane = ['Bane']

samf_medium = ['L', 'T', 'B']

In [ ]:
arealdekke_built_up = arealdekke[arealdekke['objtype'].isin(area_types)]

samferdsel_roads = samferdsel[(samferdsel['vegkategor'].isin(samf_veg_kategori) & samferdsel['medium'].isin(samf_medium) & samferdsel['objtype'].isin(samf_veg)) | (samferdsel['objtype'].isin(samf_bane) & samferdsel['medium'].isin(samf_medium))]

samferdsel_roads['geometry'] = samferdsel_roads.buffer(10)

arealdekke_vegetasjon = arealdekke[arealdekke['objtype'].isin(area_vegetasjon_types)]
arealdekke_vann = arealdekke[arealdekke['objtype'].isin(area_vann_types)]
arealdekke_open = arealdekke[arealdekke['objtype'].isin(area_open_types)]

aapen = arealdekke_open[['geometry']]
aapen['value'] = 0
aapen['color'] = 'r'

vegetasjon = arealdekke_vegetasjon[['geometry']]
vegetasjon['value'] = 1
vegetasjon['color'] = 'g'

vann = arealdekke_vann[['geometry']]
vann['value'] = 2
vann['color'] = 'b'

built_up = arealdekke[['geometry']]
built_up['value'] = 3
built_up['color'] = 'y'

roads = samferdsel_roads[['geometry']]
roads['value'] = 4
roads['color'] = 'k'

built_up.head()

In [ ]:
samferdsel_roads.objtype.unique()

In [ ]:
mask_vector = pd.concat([built_up, aapen, vegetasjon, vann, roads])
mask_vector.plot(color=mask_vector['color'])

In [ ]:
import rasterio
from rasterio import features, Affine
from rasterio.crs import CRS

meta = {'driver': 'GTiff',
        'dtype': 'float32',
        'nodata': 0,
        'width': 10240,
        'height': 10240,
        'count': 1,
        'crs': CRS.from_epsg(25832),
        'transform': Affine(10.0, 0.0, 500000.0, 0.0, -10.0, 6700000.0)}
        
with rasterio.open(f'mask.tif', 'w+', **meta) as out:
    out_arr = out.read(1)

    # this is where we create a generator of geom, value pairs to use in rasterizing
    shapes = ((geom,value) for geom, value in zip(mask_vector.geometry, mask_vector.value))

    burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)
    out.write_band(1, burned)